In [1]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from functools import reduce
import pickle
import tarfile
import numpy as np
import re
import os
import time


def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)', sent) if x.strip()]


def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true, only the sentences
    that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data


def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file,
    retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data


def vectorize_stories(data):
    inputs, queries, answers = [], [], []
    for story, query, answer in data:
        inputs.append([word_idx[w] for w in story])
        queries.append([word_idx[w] for w in query])
        answers.append(word_idx[answer])
    return (pad_sequences(inputs, maxlen=story_maxlen),
            pad_sequences(queries, maxlen=query_maxlen),
            np.array(answers))


Using TensorFlow backend.


In [2]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

challenges = {
    # QA1 with 10,000 samples
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
}
challenge_type = 'single_supporting_fact_10k'
challenge = challenges[challenge_type]

print('Extracting stories for the challenge:', challenge_type)
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))



Extracting stories for the challenge: single_supporting_fact_10k


In [3]:
# See what the data looks like

for i in range(5):
    print("Story: {}".format(' '.join(train_stories[i][0])))
    print("Query: {}".format(' '.join(train_stories[i][1])))
    print("Answer: {}".format(train_stories[i][2]))
    print("---")

Story: Mary moved to the bathroom . John went to the hallway .
Query: Where is Mary ?
Answer: bathroom
---
Story: Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden .
Query: Where is Daniel ?
Answer: hallway
---
Story: Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden . John moved to the office . Sandra journeyed to the bathroom .
Query: Where is Daniel ?
Answer: hallway
---
Story: Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden . John moved to the office . Sandra journeyed to the bathroom . Mary moved to the hallway . Daniel travelled to the office .
Query: Where is Daniel ?
Answer: office
---
Story: Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden . John moved to the office . Sandra journeyed to the bathroom . Mary mov

In [4]:
vocab = set()
for story, q, answer in train_stories + test_stories:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')


for s in list(enumerate(vocab)):
    print(s)

-
Vocab size: 22 unique words
Story max length: 68 words
Query max length: 4 words
Number of training stories: 10000
Number of test stories: 1000
-
Here's what a "story" tuple looks like (input, query, answer):
(['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'John', 'went', 'to', 'the', 'hallway', '.'], ['Where', 'is', 'Mary', '?'], 'bathroom')
-
(0, '.')
(1, '?')
(2, 'Daniel')
(3, 'John')
(4, 'Mary')
(5, 'Sandra')
(6, 'Where')
(7, 'back')
(8, 'bathroom')
(9, 'bedroom')
(10, 'garden')
(11, 'hallway')
(12, 'is')
(13, 'journeyed')
(14, 'kitchen')
(15, 'moved')
(16, 'office')
(17, 'the')
(18, 'to')
(19, 'travelled')
(20, 'went')


In [5]:
print('Vectorizing the word sequences...')
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories)

print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')


Vectorizing the word sequences...
-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (10000, 68)
inputs_test shape: (1000, 68)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (10000, 4)
queries_test shape: (1000, 4)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (10000,)
answers_test shape: (1000,)
-


In [6]:
# See individual training element.

print("Story (x): {}".format(inputs_train[0]))
print("Question (x): {}".format(queries_train[0]))
print("Answer: {}".format(answers_train[0]))

Story (x): [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  5 16 19 18  9  1  4 21 19 18 12  1]
Question (x): [ 7 13  5  2]
Answer: 9


#### Compile the Neural Network

In [7]:
print('Compiling...')

# placeholders
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))
# output: (samples, story_maxlen, embedding_dim)

# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

# the original paper uses a matrix multiplication for this reduction step.
# we choose to use a RNN instead.
answer = LSTM(32)(answer)  # (samples, 32)

# one regularization layer -- more would probably be needed.
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
print("Done.")


Compiling...
Done.


In [8]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [9]:
start_time = time.time()
# train
model.fit([inputs_train, queries_train], answers_train,
          batch_size=32,
          epochs=120,
          validation_data=([inputs_test, queries_test], answers_test))

# save
save_path = "./dnn/"
# save entire network to HDF5 (save everything, suggested)
model.save(os.path.join(save_path,"chatbot.h5"))
# save the vocab too, indexes must be the same
pickle.dump( vocab, open( os.path.join(save_path,"vocab.pkl"), "wb" ) )

elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

Train on 10000 samples, validate on 1000 samples
Epoch 1/120
10000/10000 [==============================] - 8s 791us/step - loss: 1.9370 - acc: 0.1668 - val_loss: 1.8060 - val_acc: 0.1820
Epoch 2/120
10000/10000 [==============================] - 6s 628us/step - loss: 1.7857 - acc: 0.2013 - val_loss: 1.6953 - val_acc: 0.2380
Epoch 3/120
10000/10000 [==============================] - 6s 631us/step - loss: 1.5991 - acc: 0.3489 - val_loss: 1.4960 - val_acc: 0.4120
Epoch 4/120
10000/10000 [==============================] - 6s 626us/step - loss: 1.5153 - acc: 0.3985 - val_loss: 1.4632 - val_acc: 0.4430
Epoch 5/120
10000/10000 [==============================] - 6s 638us/step - loss: 1.4908 - acc: 0.4084 - val_loss: 1.4747 - val_acc: 0.3940
Epoch 6/120
10000/10000 [==============================] - 6s 648us/step - loss: 1.4628 - acc: 0.4298 - val_loss: 1.3971 - val_acc: 0.4600
Epoch 7/120
10000/10000 [==============================] - 6s 628us/step - loss: 1.4180 - acc: 0.4513 - val_loss: 1.3

10000/10000 [==============================] - 6s 610us/step - loss: 0.6063 - acc: 0.7777 - val_loss: 0.6353 - val_acc: 0.7620
Epoch 60/120
10000/10000 [==============================] - 6s 608us/step - loss: 0.5749 - acc: 0.7926 - val_loss: 0.5780 - val_acc: 0.7870
Epoch 61/120
10000/10000 [==============================] - 6s 609us/step - loss: 0.5371 - acc: 0.8040 - val_loss: 0.5745 - val_acc: 0.7770
Epoch 62/120
10000/10000 [==============================] - 6s 609us/step - loss: 0.5015 - acc: 0.8142 - val_loss: 0.5061 - val_acc: 0.8150
Epoch 63/120
10000/10000 [==============================] - 6s 601us/step - loss: 0.4745 - acc: 0.8271 - val_loss: 0.4928 - val_acc: 0.8200
Epoch 64/120
10000/10000 [==============================] - 6s 606us/step - loss: 0.4569 - acc: 0.8288 - val_loss: 0.4491 - val_acc: 0.8250
Epoch 65/120
10000/10000 [==============================] - 6s 608us/step - loss: 0.4335 - acc: 0.8425 - val_loss: 0.4955 - val_acc: 0.8170
Epoch 66/120
10000/10000 [=======

10000/10000 [==============================] - 6s 615us/step - loss: 0.0932 - acc: 0.9680 - val_loss: 0.2030 - val_acc: 0.9470
Epoch 118/120
10000/10000 [==============================] - 6s 613us/step - loss: 0.0939 - acc: 0.9678 - val_loss: 0.2099 - val_acc: 0.9440
Epoch 119/120
10000/10000 [==============================] - 6s 603us/step - loss: 0.0903 - acc: 0.9673 - val_loss: 0.2245 - val_acc: 0.9400
Epoch 120/120
10000/10000 [==============================] - 6s 609us/step - loss: 0.0914 - acc: 0.9685 - val_loss: 0.2183 - val_acc: 0.9380
Elapsed time: 0:12:16.04


In [10]:
# Load the model, if it exists, load vocab too
save_path = "./dnn/"
model = load_model(os.path.join(save_path,"chatbot.h5"))
vocab = pickle.load( open( os.path.join(save_path,"vocab.pkl"), "rb" ) )

#### Evaluate Accuracy

We evaluate the accuracy, using the same technique as previous classification networks.

In [11]:
pred = model.predict([inputs_test, queries_test])
# See what the predictions look like, they are just probabilities of each class.
print(pred)

[[3.70085318e-16 3.93623331e-16 3.88705193e-16 ... 3.76312201e-16
  3.48653796e-16 4.17045248e-16]
 [3.00952950e-16 3.32713059e-16 3.49212732e-16 ... 3.32414930e-16
  3.49457948e-16 3.90224902e-16]
 [5.52981436e-16 5.39319271e-16 5.51997184e-16 ... 5.51572026e-16
  4.42972370e-16 5.48856068e-16]
 ...
 [1.33433333e-15 1.25757552e-15 1.37519367e-15 ... 1.32348485e-15
  1.30228033e-15 1.67511703e-15]
 [8.09931829e-18 1.04935380e-17 1.10928609e-17 ... 1.22420100e-17
  1.11964653e-17 1.20061444e-17]
 [4.23593910e-10 4.01166961e-10 4.00624062e-10 ... 3.81375792e-10
  3.79459519e-10 4.01376626e-10]]


In [12]:
# Use argmax to turn those into actual predictions.  The class (word) with the highest
# probability is the answer.

pred = np.argmax(pred,axis=1)
print(pred)

[12  9 15 12 15 12 11 12 12 17 15 11 17 10 10 15 11 15 12 12 12 17 17 12
 10 10 10 15 15 15 12 17 15 15  9 17  9 11 15 17  9 11 10 12 11 15 12  9
 17 10 10 17 11 11 15 10 15 12 10 12 12 17  9 11 11 15 15  9 17 10 11 12
 11 10  9 15  9 15 12 17 11  9 10 11 17 11 11 12 15 15  9  9 12  9  9 15
 15 15 10 10 10  9 17 11 11 12 10  9 17 10  9  9 15 12 15 11 17 15  9 17
 10  9  9 12 10 17 11 11 11  9 11 11  9 11 11  9 17 15  9  9 17 12 12 17
 10 15 15 15 15 11 11 11  9 17 10  9 11 10 17 12 10 10 10 11  9 11 11 11
 15 10 17 17 11  9  9 17 15 10 17 17 12 12 12 12 10  9 17 11 15 17 12 15
 11 12 12 11 17 15  9 10 12 17 12 10 12  9 15 12  9  9  9 10 12 10 10 10
  9 12 12  9  9 12 15  9 10 17 11 11  9  9 17 11 12 17 11 10 10  9 10 17
 12 15 11 11 15 15 11 11 12 11 10 17  9  9 10 10 17 17 15 11 11 17 12 17
 17  9 11 10  9 10 15 17 15 15  9 17 15 12 17  9  9 11 17 17 11 17  9 12
 11  9 15 15 10 15 15 11 11 12 12  9  9  9 15 17 12 11  9 17  9 15 11 11
 17 10 11 17 10 17 17 11 11 15 15 11 11  9 10 11 11

In [13]:
score = metrics.accuracy_score(answers_test, pred)
print("Final accuracy: {}".format(score))

Final accuracy: 0.938


In [16]:
print("Remember, I only know these words: {}".format(vocab))
print()
story = "Daniel went to the hallway. Mary went to the bathroom.  Daniel went to the bedroom."
query = "Where is Sandra?"

adhoc_stories = (tokenize(story), tokenize(query), '?')

adhoc_train, adhoc_query, adhoc_answer = vectorize_stories([adhoc_stories])

pred = model.predict([adhoc_train, adhoc_query])
print(pred[0])
pred = np.argmax(pred,axis=1)
print("Answer: {}({})".format(vocab[pred[0]-1],pred))

Remember, I only know these words: ['.', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'Where', 'back', 'bathroom', 'bedroom', 'garden', 'hallway', 'is', 'journeyed', 'kitchen', 'moved', 'office', 'the', 'to', 'travelled', 'went']

[1.14450151e-10 1.07695346e-10 1.21892912e-10 1.33932684e-10
 1.35364442e-10 1.08836461e-10 1.11957887e-10 9.79306289e-11
 1.27800437e-10 5.32004706e-05 6.92148285e-04 2.28924230e-01
 5.23171723e-01 9.91292395e-11 1.08987687e-10 1.38701946e-01
 1.34748782e-10 1.08456723e-01 1.12304145e-10 1.17121590e-10
 1.03777230e-10 1.33520237e-10]
Answer: hallway([12])
